In [20]:
import numpy as np
import mne
from mne.io import read_raw_fif
from mne import Annotations, Epochs, events_from_annotations

In [2]:
raw = read_raw_fif("/home/gemeinl/code/fake_data/0_raw.fif")

Opening raw data file /home/gemeinl/code/fake_data/0_raw.fif...
Isotrak not found
    Range : 0 ... 48912 =      0.000 ...   489.120 secs
Ready.


In [3]:
fs = raw.info["sfreq"]
duration = raw.n_times / fs

In [4]:
raw.annotations

<Annotations  |  0 segments , orig_time : None>

create an annotaiton for whole trial which holds for all cases of cropping

In [18]:
trial_annotations = Annotations(onset=np.array([0]), duration=duration, description=np.array("trial"))
raw.set_annotations(trial_annotations)

<Raw  |  0_raw.fif, n_channels x n_times : 21 x 48913 (489.1 sec), ~45 kB, data not loaded>

create a single crop for whole trial

In [19]:
trial_events, trial_event_ids = events_from_annotations(raw, chunk_duration=duration)
trial_epochs = Epochs(raw, trial_events, event_id=trial_event_ids, tmin=0, tmax=raw.times[-1], baseline=None)

Used Annotations descriptions: ['trial']
1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated


In [20]:
trial_epochs.get_data().shape

Loading data for 1 events and 48913 original time points ...
0 bad epochs dropped


(1, 21, 48913)

__chunk_duration is the gap between two consecutive crops (crop_size - overlap)__

In [101]:
crop_size = 1
chunk_duration = 1

In [102]:
crop_events, crop_event_ids = events_from_annotations(raw, chunk_duration=chunk_duration/fs)
last_valid_crop_start = raw.n_times - crop_size
crop_events = crop_events[crop_events[:,0] < last_valid_crop_start]
# what label to put for the last imcomplete / start shifted crop?
crop_events = np.concatenate((crop_events, [[last_valid_crop_start, 0, 1]]))

Used Annotations descriptions: ['trial']


In [105]:
crop_events

array([[    0,     0,     1],
       [    1,     0,     1],
       [    2,     0,     1],
       ...,
       [48910,     0,     1],
       [48911,     0,     1],
       [48912,     0,     1]])

In [103]:
crop_epochs = Epochs(raw, crop_events, event_id=crop_event_ids, tmin=0, tmax=(crop_size-1)/fs, baseline=None)

48913 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated


In [104]:
crop_epochs.get_data().shape

Loading data for 48913 events and 1 original time points ...
0 bad epochs dropped


(48913, 21, 1)

actually there is one mne function that does everything we want:

In [5]:
from mne import make_fixed_length_events

In [6]:
overlap_samples = 199
crop_size_samples = 200

In [7]:
crop_events = make_fixed_length_events(raw, id=1, duration=crop_size_samples/fs, overlap=overlap_samples/fs)

In [8]:
last_valid_crop_start = raw.n_times - crop_size_samples
crop_events = crop_events[crop_events[:,0] < last_valid_crop_start]
# what label to put for the last imcomplete / start shifted crop?
crop_events = np.concatenate((crop_events, [[last_valid_crop_start, 0, 1]]))

In [9]:
crop_epochs = Epochs(raw, crop_events, tmin=0, tmax=(crop_size_samples-1)/fs, baseline=None)

48714 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated


In [10]:
crop_epochs.get_data().shape

Loading data for 48714 events and 200 original time points ...
0 bad epochs dropped


(48714, 21, 200)

for regression to continuous labels (with same sampling frequency as data)

In [14]:
info = mne.create_info(["label"], sfreq=100)
raw_labels = RawArray(np.random.choice(4, len(raw)).reshape(1, -1), info)

Creating RawArray with float64 data, n_channels=1, n_times=48913
    Range : 0 ... 48912 =      0.000 ...   489.120 secs
Ready.


In [15]:
crop_labels = Epochs(raw_labels, crop_events, tmin=0, tmax=(crop_size_samples-1)/fs, baseline=None)

48714 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated


In [18]:
labels = crop_labels.get_data()

Loading data for 48714 events and 200 original time points ...


In [19]:
labels.shape

(48714, 1, 200)